![](https://www.skygrid.com/wp-content/uploads/2020/04/package-delivery.png) 

## Problem understanding 

The Internet has profoundly changed the way we buy things, but the online shopping of today is likely not the end of that change; after each purchase we still need to wait multiple days for physical goods to be carried to our doorstep. This is where drones come in ­ autonomous, electric vehicles delivering online purchases. Flying, so never stuck in traffic. As drone technology improves every year, there remains a major issue: how do we manage and coordinate all those drones?

## Task

Given a hypothetical fleet of drones, a list of customer orders and availability of the individual products in warehouses, the task is to schedule the drone operations so that the orders are completed as soon as possible.

Description of output variables (defined according to the Hashcode instructions "File Format"):
- grid_row, int, - number of rows in the grid
- grid_col, int, - number of columns in the grid
- n_drones, int, - number of drones available
- max_turns, int, - maximum length of the simulation in "turns"
- max_payload, int, - maximum load that a drone can carry

- n_prod_types, int, P - total number of different product types available in wharehouses
- weight_prod_types, int - list of len P, weight of each of the different product types.
- n_wrhs, int, - total number of warehouses

- wrhs_info, int list of len n_whrs, - each element [whrs_loc, num_itms_per_prodtype] of the array contains the location of the warehouse and the number of items of each product type in the warehouse.

 *Example: the first warehouse
 wrhs_info[0] = [[113, 179], [0, 0, 5, 1, 0, 0, 0, 0, 2, 0, 4, 0, 0, 0, 0, 8, 11, 5, 0, ...]]*

- n_orders, int, - total of number of order to be completed.
- order_info, int - list of len n_orders, each element [order_loc, n_order_items, prod_type_of_prod_item] of the array contains the location of the order, the number of order product items and finally the the product types of the product items.

 *Example: the first order
 order_info[0] = [[340, 371], [8], [226, 183, 6, 220, 299, 280, 12, 42]]*



In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

In [ ]:
def sort_data(text):
        f = open(text, "r")
        lines = f.readlines()
        f.close()

        grid_row, grid_col, n_drones, max_turns, max_payload = [int(lines[0].split()[i]) for i in range(5)]
        n_prod_types = int(lines[1])
        weight_prod_types = lines[2].split()
        weight_prod_types = [int(i) for i in weight_prod_types]
        n_wrhs = int(lines[3])

        wrhs_info = []
        i=4
        while(i<n_wrhs*2+4):
            wrhs_loc, num_itms_per_prodtype = lines[i].split(), lines[i+1].split()
            wrhs_loc = [int(i) for i in wrhs_loc]
            num_itms_per_prodtype = [int(i) for i in num_itms_per_prodtype]
            wrhs_info.append([wrhs_loc, num_itms_per_prodtype])
            i = i+2

        n_orders = int(lines[24])
        i=25
        order_info = []
        while(i<n_orders*3+25):
            order_loc, n_order_items, prod_type_of_prod_item =  lines[i].split(), lines[i+1].split(), lines[i+2].split()
            order_loc = [int(i) for i in order_loc]
            n_order_items = [int(i) for i in n_order_items]
            prod_type_of_prod_item = [int(i) for i in prod_type_of_prod_item]
            order_info.append([order_loc, n_order_items, prod_type_of_prod_item])
            i = i+3
        return grid_row, grid_col, n_drones, max_turns, max_payload, n_prod_types, weight_prod_types, n_wrhs, wrhs_info, n_orders, order_info
    
sns.set()

data = sort_data('/kaggle/input/hashcode-drone-delivery/busy_day.in')
grid_row, grid_col, n_drones, max_turns, max_payload, n_prod_types, weight_prod_types, n_wrhs, wrhs_info, n_orders, order_info = [data[i] for i in range(11)]

## EDA

In [ ]:
#-- Geograhical plot of warehouses and orders
wrhs_loc = [wrhs_info[i][0] for i in range(len(wrhs_info))]
order_loc = [order_info[i][0] for i in range(len(order_info))]

plt.figure()
plt.title("Location of orders and warehouses")
for i in order_loc:
    plt.plot(i[0], int(i[1]), "-x", alpha = 0.8)
for i in wrhs_loc:
    plt.plot(i[0], int(i[1]), "-o", color = 'green', ms = '10')
plt.show()

#-- Number of items in each warehouse
wrhs_n_items = [sum(i[1]) for i in wrhs_info]
plt.figure()
plt.title("Number of Items in each warehouse")
plt.bar(range(len(wrhs_info)), wrhs_n_items)
plt.show()

## General approach

The main idea of the algorithm is the following:\
1) Each drone finds the nearest warehouse and goes there\
2) Next, the drone finds the nearest order whose products are available at the current warehouse and gets assigned to that order\
3) The drone takes the products available at the warehouse and if it has some spare space ('remainder') it looks for another nearest warehouse with the products needed for the current order. If there's such a warehouse then the drone goes there to pick up the missing products types\
4) The drone delivers the order (the order may be still incomplete) and again looks for the nearest warehouse

## Definition of classes

In [ ]:
class Dataframes():

    def __init__(self):
        data = sort_data('/kaggle/input/hashcode-drone-delivery/busy_day.in')
        self.grid_row, \
        self.grid_col, \
        self.n_drones, \
        self.max_turns, \
        self.max_payload, \
        self.n_prod_types, \
        self.weight_prod_types, \
        self.n_wrhs, \
        self.wrhs_info, \
        self.n_orders, \
        self.order_info = [data[i] for i in range(11)]

    def get_df_orders(self):
        """ Returns a dataframe with orders location and products needed."""
        x_order_loc = [self.order_info[i][0][0] for i in range(len(self.order_info))]
        y_order_loc = [self.order_info[i][0][1] for i in range(len(self.order_info))]
        n_items_per_order = [self.order_info[i][1][0] for i in range(len(self.order_info))]
        df_orders = pd.DataFrame(list(zip(x_order_loc, y_order_loc, n_items_per_order)),
                                 columns=["X", "Y", "N of Items"])
        return df_orders

    def get_df_wrhs(self):
        """ Returns a dataframe with warehouses location and products available."""
        wrhs_x = [self.wrhs_info[i][0][0] for i in range(len(self.wrhs_info))]
        wrhs_y = [self.wrhs_info[i][0][1] for i in range(len(self.wrhs_info))]
        n_items_per_product_type = [self.wrhs_info[i][1] for i in range(len(self.wrhs_info))]
        df_wrhs = pd.DataFrame(list(zip(wrhs_x, wrhs_y, n_items_per_product_type)),
                               columns=["X", "Y", "Amounts"])
        return df_wrhs

# -- Dataframe for Warehouses
data = Dataframes()
df_wrhs = data.get_df_wrhs()
print("-----------------------Warehouses dataframe---------------------")
print('\n', df_wrhs, '\n')

# -- Dataframe for Orders
df_orders = data.get_df_orders()
print("-----------------------Orders dataframe--------------------------")
print('\n', df_orders)

In [ ]:
def dist(a, b):
    """Calculates distance between two or more objects."""
    if isinstance(a, np.ndarray) or isinstance(b, np.ndarray):
        return np.sqrt(((a - b) ** 2).sum(1))
    return np.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

def num_turns(self, dist):
    """Calculate number of turns required to cover a certain distance."""
    return math.ceil(dist)

In [ ]:
class Drone():

    def __init__(self, num, weight_prod_types):
        self.num = num # number of the drone
        self.pld_weight = 0 # current drone payload
        self.cur_pos = [0, 0] # current position of a drone
        self.turns = 0 # initializing number of turns to zero
        self.actions = []  # list of drone actions 
        self.state = 'W' # initialize drone state to 'wait'
        self.weights = np.array(weight_prod_types) 
        self.orders = [] # list of orders that a drone is working on 
        self.amounts = np.zeros(400)
        self.types = np.arange(400)
        self.remainder = 0 # spare payload of a drone after loading products 

    def __repr__(self):
        return '(num: ' + str(self.num) + ', ' + 'types: ' + str(self.types[self.amounts>0]) + ', ' + 'amounts: ' +  str(self.amounts[self.amounts>0]) + ')'
    
    def compute_weight(self):
        """Returns total weight of all items carried by the drone at the moment."""
        return self.weights[self.types]*self.amounts[self.types]

    def load(self, prod_types, prod_qnty, wrhs):
        """Loads products into a drone."""
        self.state = 'L' 
        self.amounts[prod_types] += 1
        self.compute_weight()
        message = []
        for i in range(prod_types.shape[0]):
            message.append(f'{self.num} {self.state} {wrhs.num} {prod_types[i]} {prod_qnty[i]}')
        self.turns += prod_types.shape[0] 
        return message

    def unload(self, prod_types, qnty):
        self.turns += 1

    def deliver(self, prod_types, prod_qnty, order, orders):
        """Delivers products to the order (unloading process)."""
        self.state = 'D'
        message = []
        for i in range(prod_types.shape[0]):
            message.append(f'{self.num} {self.state} {order.num} {prod_types[i]} {prod_qnty[i]}')
        self.turns += prod_types.shape[0]
        self.amounts[prod_types] -= prod_qnty
        self.compute_weight()
        return message 

    def wait(self, n_turns):
        """Drone waits for n turns."""
        self.state = 'W'
        self.turns += n_turns

    def get_cur_pos(self):
        """Returns current position of a drone."""
        return self.cur_pos

    def update_cur_pos(self, new_pos):
        self.turns += np.int(np.ceil(dist(self.cur_pos, new_pos)))
        self.cur_pos = new_pos 

    def find_nearest_wh(self, warehouses):
        """Returns the nearest warehouse."""
        wh = np.array(warehouses.positions, dtype=np.float64)
        wh[(warehouses.check_empty())|warehouses.not_avail] = np.inf
        d = dist(self.cur_pos, wh)
        return warehouses.dict[np.argmin(d)]

    def check_pld_weight(self):
        """Checks if the drone is full."""
        return self.pld_weight <= 200

    def select_avail_types(self, wrhs, order):
        """Returns order product types which are available at the warehouse."""
        avail_types = order.prod_types[order.check_avail_types(wrhs)]
        return avail_types

    def select_avail_quantities(self, avail_types, order, wrhs):
        """Returns the minimum quantity between that available in the warehouse and the one required in order."""
        wrhs_qnty = wrhs.amounts[avail_types] 
        order_qnty = order.amounts[avail_types]
        selected_qnty = np.column_stack((wrhs_qnty, order_qnty)).min(1)
        assert np.all(selected_qnty == np.min((wrhs_qnty, order_qnty), 0))
        return selected_qnty

    def find_nearest_order(self, orders, warehouses, wrhs):  # dictionary of orders (class Orders)
        """Finds the nearest order to the warehouse. """
        if self.orders != []:
            last_order = self.orders[-1]
            if last_order.amount>0 and np.any(last_order.check_avail_types(wrhs)):
                return self.orders[-1], 'last' #54229
        order_pos = orders.positions.astype(np.float64)
        c = orders.completed
        check_avail = warehouses.all_avail_orders[wrhs.num]
        
        if c.sum() == 1250:
            return 'All orders are completed', 'completed'
        order_pos[(c)|(~check_avail)] = np.inf
        d = dist(self.cur_pos, order_pos)
        
        if np.min(d) == np.inf:
            order_pos = orders.positions.astype(np.float64)
            check_avail = warehouses.any_avail_orders[wrhs.num]
            order_pos[(c)|(~check_avail)] = np.inf
            d = dist(self.cur_pos, order_pos)
            if np.min(d) == np.inf:
                assert order_pos.min() == np.inf
                wrhs.update_not_avail(warehouses)
            nearest_order = orders.dict[np.argmin(d)]
            return nearest_order, 0
        nearest_order = orders.dict[np.argmin(d)]
        return nearest_order, 1

    def assign_order(self, order, wrhs, warehouses, orders):
        """Assigns an order to the drone: loads the products into the drone 
           and removes them from the warehouse."""
        self.orders.append(order)
        avail_types = self.select_avail_types(wrhs, order)
        avail_qnty = self.select_avail_quantities(avail_types, order, wrhs)
        
        if np.sum(self.weights[avail_types]*avail_qnty) <= 200:
            # if total weight of the order is below 200, the drone takes all the products needed
            new_types = avail_types
            new_qnty = avail_qnty
            payload = self.weights[new_types].sum()

        else:
            # else, the products are sorted in descending order by weight and the drone picks 
            # the items until the weight exceeds 200. The remainder space is counted. 
            types = np.repeat(avail_types, avail_qnty)
            weights = self.weights[types]
            types_sorted = types[weights.argsort()]
            mask_le_200 = self.weights[types_sorted].cumsum() <= 200
            new_types_repeated = types_sorted[mask_le_200][::-1]
            new_types, new_qnty = np.unique(new_types_repeated, return_counts=True)
            payload = self.weights[new_types_repeated].sum()

            
        wrhs.remove_product(new_types, new_qnty, warehouses)
        loading_message = self.load(new_types, new_qnty, wrhs)

        remainder = 200 - payload
        self.remainder = remainder
        loading_message_nono = []
        result_nono = []
        if remainder>0:
            order_neighbors = orders.neighbors[order.num]
            if np.any(order_neighbors):
                orders_near_order = orders.array[order_neighbors]
                not_completed = ~orders.completed[order_neighbors]
                check_types = np.array([np.any(o.check_avail_types(wrhs)) for o in orders_near_order])
                check_weights = np.array([np.any((self.weights[o.prod_types]<=remainder)) for o in orders_near_order])
                mask = not_completed & check_types & check_weights
                orders_near_order = orders_near_order[mask]
                
                if orders_near_order.shape[0]>0:
                    d = [o.amount for o in orders_near_order]
                    nono = orders_near_order[np.argmax(d)] #nearest order to the nearest order 
                    avail_types_nono = nono.prod_types[wrhs.avail_products[nono.prod_types]]
                    avail_qnty_nono = self.select_avail_quantities(avail_types_nono, nono, wrhs)
                    types_nono = np.repeat(avail_types_nono, avail_qnty_nono)
                    weights_nono = self.weights[types_nono]
                    sorted_types_nono = types_nono[weights_nono.argsort()]
                    weights_nono.sort()
                    mask_le_rem = weights_nono.cumsum() <= remainder
                    new_types_repeated_nono = sorted_types_nono[mask_le_rem]
                    new_types_nono, new_qnty_nono = np.unique(new_types_repeated_nono, return_counts=True)
                    wrhs.remove_product(new_types_nono, new_qnty_nono, warehouses)
                    loading_message_nono = self.load(new_types_nono, new_qnty_nono, wrhs)
                    result_nono = new_types_nono, new_qnty_nono, nono
        return new_types, new_qnty, loading_message + loading_message_nono, result_nono

    def deliver_order(self, types, qnty, order, orders):
        """Delivers the products to the order and checks if the order was completed."""
        self.update_cur_pos(order.position)
        delivery_message = self.deliver(types, qnty, order, orders)
        order.remove_prod(types, qnty)
        assert order.amount >= 0
        self.remainder = 0
        self.compute_weight()
        order.check_completed(self.turns, orders)
        return delivery_message

    def find_nearest_wh_with_types(self, warehouses, leftover_types):
        """Returns the nearest warehouse with the products needed."""
        leftover_acceptable_types = leftover_types[self.weights[leftover_types]<=self.remainder]
        avail_acceptable_leftover = np.array([np.any(x[leftover_acceptable_types]) for x in warehouses.avail_products])
        wh = np.array(warehouses.positions, dtype=np.float64)
        wh[~avail_acceptable_leftover] = np.inf
        d = dist(self.cur_pos, wh)
    
        if d.min() == np.inf:
            return 'no_pickup', []
        index_argmin = np.argmin(d)
        wh_next_pickup = warehouses.dict[index_argmin]
        types_in_remainder = leftover_types[wh_next_pickup.avail_products[leftover_types]]
        types_sorted = types_in_remainder[self.weights[types_in_remainder].argsort()]
        types_chosen = types_sorted[self.weights[types_sorted].cumsum()<=self.remainder]
        return wh_next_pickup, types_chosen  

    def assign_pickup(self, wh_next_pickup, types_in_remainder, warehouses):
        """Picks the remaining products of the order from another warehouse."""
        qnty_remainder = np.ones(len(types_in_remainder), dtype=np.int64)
        wh_next_pickup.remove_product(types_in_remainder, qnty_remainder, warehouses)
        loading_message = self.load(types_in_remainder, qnty_remainder, wh_next_pickup)
        assert self.compute_weight().sum() <= 200
        self.remainder -= self.weights[types_in_remainder].sum()
        return types_in_remainder, qnty_remainder, loading_message

In [ ]:
class Order():

    def __init__(self, num, x, y, amount, types, weight_prod_types):
        self.amount = amount[0]
        self.num = num
        self.position = (x, y)
        self.completed = False
        self.turn_order_completed = 0
        self.all_weights = weight_prod_types
        self.typelist = np.array(types)
        self.amounts = np.zeros(400, dtype=np.int32) 
        self.types = np.arange(400, dtype=np.int32)
        t, a = np.unique(self.typelist, return_counts=True)
        self.amounts[t] += a
        self.weights = np.array(weight_prod_types)
        self.prod_amounts = self.amounts[self.amounts>0]
        self.prod_types = self.types[self.amounts>0]
        self.tot_weight = np.sum(self.weights*self.amounts)
        self.assigned = 0

    def __repr__(self):
        return '(num: ' + str(self.num) + ', ' + 'n_items: ' + str(
            self.amount) + ', ' + 'tot_weight: ' + str(
                self.tot_weight) + ', ' + 'types: ' + str(self.prod_types) + ', ' + 'quantities: ' + str(
                    self.prod_amounts) + ', ' + 'weights:' + str(self.weights[self.amounts>0]) + ')'

    def remove_prod(self, prod_type, prod_qnty):
        """Removes a product from an order once it is delivered."""
        self.amounts[prod_type] -= prod_qnty
        self.prod_amounts = self.amounts[self.amounts>0]
        self.prod_types = self.types[self.amounts>0]
        t = list(self.typelist)
        for x in list(np.repeat(prod_type, prod_qnty)):
            t.remove(x)
        self.typelist = np.array(t)
        self.amount -= prod_qnty.sum()
        self.tot_weight = np.sum(self.weights[self.prod_types]*self.amounts[self.prod_types])
        
    def check_completed(self, turn, orders):
        """Checks if the order is completed and saves the turn in which it was completed."""
        if self.amounts.sum() == 0:
            self.completed = True
            self.turn_order_completed = turn
            orders.add_completed(self.num)
            orders.turn_order_completed[self.num] = self.turn_order_completed

    def check_avail_types(self, wrhs):
        """"Returns a boolean list for product types available at the warehouse."""
        avail = wrhs.avail_products[self.prod_types]
        return avail

class Orders():
    """Auxiliary class for orders which stores positions and states of all the orders."""
    def __init__(self, n_orders, ordersdict):
        positions = [ordersdict[x].position for x in ordersdict]
        self.positions = np.array(positions)
        self.num = np.array([o.num for o in ordersdict.values()])
        self.array = np.array([o for o in ordersdict.values()])
        pos = np.array(self.positions)
        neighbors = (np.sum((pos.reshape((1,1250,2)) - pos.reshape((1250, 1, 2)))**2, 2)<10)
        ni = np.diag_indices(1250)
        neighbors[ni] = False
        self.neighbors = neighbors
        self.n_orders = n_orders
        self.dict = ordersdict
        completed = [ordersdict[x].completed for x in ordersdict]
        self.completed = np.array(completed)
        self.turn_order_completed = [ordersdict[x].turn_order_completed for x in ordersdict]

    def __repr__(self):
        return f'n_orders: {self.n_orders}, completed: {self.completed.sum()}'

    def add_completed(self, ordernum):
        self.completed[ordernum] = True

In [ ]:
class Product():

    def __init__(self, type, weight):
        self.type = type
        self.weight = weight

In [ ]:
class Warehouse():

    def __init__(self, num, x, y, amounts, weight_product_types):
        # amounts - list of every product type amount, some are zero if that product type is not available
        # weight_product_types - weights of product types in this warehouse
        self.num = num
        self.position = (x, y) 
        self.amounts = np.array(amounts)
        self.types = np.arange(400)
        self.avail_products = (self.amounts>0)
        self.tot_amounts = self.amounts.sum()
        self.not_avail = False
        
    def __repr__(self):
        return '(num: ' + str(self.num) + ', ' + 'position: ' + str(self.position) + ', ' + 'tot_amount: ' + str(self.tot_amounts) + ')'

    def add_product(self, prod_type, prod_qnty):
        """Adds products to the warehouse."""
        self.prod_amounts.append(prod_qnty)

    def remove_product(self, prod_type, prod_qnty, warehouses):
        """Removes a certain quantity of a product type from a warehouse."""
        self.amounts[prod_type] -= prod_qnty
        self.avail_products = (self.amounts>0) 
        warehouses.tot_amounts[self.num] = self.amounts.sum()
        warehouses.avail_products[self.num] =  self.avail_products
        assert np.all(self.amounts>=0)
    
    def update_not_avail(self, warehouses):
        self.not_avail = True
        warehouses.not_avail[self.num] = True

    # only update the true ones
    def update_availability(self,  warehouses, orders):
        """For each warehouse updates the list of orders with all/any products available
           at the warehouse."""
        not_completed_orders = np.flatnonzero(~orders.completed)
        warehouses.all_avail_orders[self.num][orders.completed] = False
        warehouses.any_avail_orders[self.num][orders.completed] = False
        avail_orders_any = np.flatnonzero(warehouses.any_avail_orders[self.num])
        warehouses.all_avail_orders[self.num][not_completed_orders] = np.array([np.all(orders.dict[o].check_avail_types(self)) 
        for o in not_completed_orders]) 
        warehouses.any_avail_orders[self.num][avail_orders_any] = np.array([np.any(orders.dict[o].check_avail_types(self)) 
        for o in avail_orders_any]) 
                
class Warehouses():
    """Auxiliary class for warehouses which stores positions of all the warehouses. 
       For each warehouse a dictionary of orders which have all products available at 
       that warehouse and another dictionary of orders which have any of their products available at the warehouse."""
    def __init__(self, n_wrhs, orders, wrhsdict):
        self.n_wrhs = n_wrhs
        self.dict = wrhsdict
        positions = [wrhsdict[x].position for x in wrhsdict]
        self.tot_amounts = [wrhsdict[x].amounts.sum() for x in wrhsdict]
        self.positions = positions
        self.avail_products = [wrhsdict[x].avail_products for x in wrhsdict]
        self.not_avail = np.array([(wrhsdict[x].not_avail) for x in wrhsdict])
        self.all_avail_orders = {}
        for w in self.dict.values():
            self.all_avail_orders[w.num] = np.array([np.all(
                o.check_avail_types(w)) for o in orders.dict.values()])
        self.any_avail_orders = {}
        for w in self.dict.values():
            self.any_avail_orders[w.num] = np.array([np.any(
                o.check_avail_types(w)) for o in orders.dict.values()])

    def check_empty(self):
        empty_warehouses = [self.dict[x].amounts.sum() <= 0 for x in self.dict]
        return np.array(empty_warehouses)


## Simulation

In [ ]:
# DRONES
drones = [Drone(i, weight_prod_types) for i in range(n_drones)]
dronesdict = dict(enumerate(drones))

# ORDERS
orderslist = [Order(i, order_info[i][0][0], order_info[i][0][1], order_info[i][1], order_info[i][2], weight_prod_types) for
          i in range(n_orders)]
ordersdict = dict(enumerate(orderslist))
orders = Orders(n_orders, ordersdict)

# WAREHOUSES
wrhslist = [Warehouse(i, wrhs_info[i][0][0], wrhs_info[i][0][1], wrhs_info[i][1], weight_prod_types) for i in
        range(n_wrhs)]
wrhsdict = dict(enumerate(wrhslist))
warehouses = Warehouses(n_wrhs, orders, wrhsdict)

# Drones must start a wharehouse 0
# then move 3 drones to each warehouse
for i in dronesdict:
    dronesdict[i].update_cur_pos(wrhsdict[0].position)
    dronesdict[i].update_cur_pos(wrhsdict[i%10].position)

completed = 0
message = 0
no_type = 0
remainder = 0
total_message = []
drone_turns = np.array([x.turns for x in dronesdict.values()])

while completed<1251:
    loading_message_r = []
    drone = dronesdict[np.argmin(drone_turns)]
    nearest_warehouse = drone.find_nearest_wh(warehouses)
    drone.update_cur_pos(nearest_warehouse.position)
        
    nearest_warehouse.update_availability(warehouses, orders)
    nearest_order, all = drone.find_nearest_order(orders, warehouses, nearest_warehouse)
    if nearest_order == 'All orders are completed':
        message = 'DONE'
        break
    types, qnty, loading_message, nono = drone.assign_order(nearest_order, nearest_warehouse, warehouses, orders)
    nearest_warehouse.update_availability(warehouses, orders)
    if types.shape[0]>0:
        delivery_message = drone.deliver_order(types, qnty, nearest_order, orders)
        print(f'completed : {orders.completed.sum()}', f'turn: {drone.turns}')
        if len(nono)>0:
            types_nono, qnty_nono, order_nono = nono
            delivery_message_nono = drone.deliver_order(types_nono, qnty_nono, order_nono, orders)
            delivery_message += delivery_message_nono
        total_message.append(loading_message + delivery_message)
        assert len(loading_message) == len(delivery_message)
    else: 
        print('no_type')
        no_type += 1
    drone_turns[drone.num] = drone.turns
    completed = orders.completed.sum()
if message == 'DONE':
    print(f'warehouses: {warehouses.dict}')
    final_message = []
    for x in total_message:
        final_message.extend(x)
    n_lines = len(final_message)
#     print(f'message: {total_message}')
    max_turns_drones = np.max(drone_turns)
    print(f'max number of turns: {max_turns_drones}') 
    print(f'number of cycles with 0 products delivered: {no_type}')
    turns_orders_completed = np.array(orders.turn_order_completed)
    score = np.sum(np.ceil(((max_turns_drones-turns_orders_completed)/max_turns_drones)*100))
    print(f'score: {score}')

In [ ]:
pd_message = pd.concat((pd.Series(n_lines), pd.Series(final_message)), ignore_index = True)
pd_message.to_csv('submission.csv', index = False, header = False)